<span style='font-size:2.4em'>Procesamiento masivo de datos con SparkR</span>

<span style='font-size:1.5em'>VIII Jornadas de usuarios de R. Albacete, Castilla-La Mancha, 17 y 18 de noviembre de 2016</span>

Taller impartido por: <span style='font-size:1.2em'>Manuel Jesús Parra Royón</span>


![Alt](https://sites.google.com/site/manuparra/home/logoparty.png)

<HR>

# Operaciones sobre SparkDataFrames

![Spark+R](https://sites.google.com/site/manuparra/home/SparkRlogo.png)

Como siempre para todos nuestros `scripts` con **SparkR**, cargamos la biblioteca, y creamos una nueva sesión de SparkR.

En este caso:

<span style="background-color:red;color:white">&nbsp; &nbsp; Cuidado con la cantidad de MEMORIA que usamos para esta sección ! &nbsp; &nbsp; </span>

In [2]:
.libPaths(c(file.path(Sys.getenv("SPARK_HOME"),"R/lib/"),.libPaths()))
library(SparkR)
sparkR.session(appName="EntornoInicio", master = "local[*]", sparkConfig = list(spark.driver.memory = "1g"))


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union

Spark package found in SPARK_HOME: /usr/local/spark/


Launching java with spark-submit command /usr/local/spark//bin/spark-submit   --driver-memory "1g" sparkr-shell /tmp/RtmpTN6Bd7/backend_portaf810f87661 


Java ref type org.apache.spark.sql.SparkSession id 1 

Los ``SparkDataFrames`` soportan un alto número de funciones para hacer un procesado de datos estructurado. 

Vamos a poner en práctica las más utilizadas. La lista completa de operaciones que se pueden aplicar se puede ver desde API de SparkR en https://spark.apache.org/docs/latest/api/R/index.html

![funcSparkR](https://sites.google.com/site/manuparra/home/functionSparkR.jpg)


# Operaciones con SparkDataFrames

Cargamos un conjunto de datos masivo desde el repositirio de datasets.


Podemos verlos desde: http://localhost:25980/tree


El dataset que vamos a usar para el procesamiento de dato masivos, corresponde con un conjunto de datos de los registros de viaje en *TAXI*, donde se capturan las fechas y horas de recogida y devolución de pasajeros, lugares de recogida y entrega (coordenadas), distancias de viaje, tarifas detalladas, tipos de tarifas, tipos de pago y conteos de pasajeros que van en el taxi.  

El dataset tiene **MUCHAS** posibilidades de procesamiento y también extracción de conocimiento.

Estos conjuntos de datos adjuntos fueron recopilados y proporcionados por la Comisión de Taxisde Nueva York (TLC) http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml.


![Alt](http://www.nyc.gov/html/tlc/images/features/fi_about_photo_trip_records.png)



**Características del conjunto de datos original:**

- El conjunto de datos NYCTaxiTrips en total tiene sobre **267GB**, que pueden ser manejados sin problema por SparkR (en un cluster real, no sobre una máquina virtual sencilla). 
- En total contiene 1100 millones de registros.
- Más información de como se gestionan 1100 millones de instancias en la siguiente web y se soluciona este problema  problema real: http://toddwschneider.com/posts/analyzing-1-1-billion-nyc-taxi-and-uber-trips-with-a-vengeance/


Más datasets masivos de NYCTaxiTrips en:  http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml


Primero revisamos los distintos dataset que se han preparado en: http://localhost:25980/tree/datasets

* yellow_tripdata_2016-01.csv
* yellow_tripdata_2016-02_small1.csv
* yellow_tripdata_2016-02_small2.csv
* yellow_tripdata_2016-02_small3.csv

In [3]:
# Cargamos una versión reducida de los datos en CSV
df_nyctrips <- read.df("/root/TallerSparkR/datasets/yellow_tripdata_2016-02_small3.csv", "csv", header = "true", inferSchema = "true")

Estudiamos de manera superficial el dataset

In [4]:
# Comprobamos los campos del dataset
printSchema(df_nyctrips)

# Comprobamos como son los datos:
head(df_nyctrips)

# Contamos el total del registros:
count(df_nyctrips)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-02-25 17:24:20,2016-02-25 17:27:20,2,0.70,-73.94725,40.76377,1,N,-73.99201,40.73539,2,5.0,0.0,0.5,0.0,0,0.3,5.8
2,2016-02-25 23:10:50,2016-02-25 23:31:50,2,5.52,-73.98302,40.75099,1,N,-73.98859,40.75884,2,20.0,0.5,0.5,0.0,0,0.3,21.3
2,2016-02-01 00:00:01,2016-02-01 00:10:52,6,1.99,-73.99234,40.75820,1,N,-73.96436,40.75798,1,9.5,0.5,0.5,0.7,0,0.3,11.5
1,2016-02-01 00:00:04,2016-02-01 00:05:16,1,1.50,-73.98145,40.74972,1,N,-73.98232,40.76398,2,6.5,0.5,0.5,0.0,0,0.3,7.8
2,2016-02-01 00:00:05,2016-02-01 00:20:59,1,5.60,-74.00060,40.72976,1,N,-73.95132,40.66983,1,20.0,0.5,0.5,4.0,0,0.3,25.3
2,2016-02-01 00:00:06,2016-02-01 00:15:01,1,4.69,-74.00510,40.71901,1,N,-73.94709,40.68888,2,16.0,0.5,0.5,0.0,0,0.3,17.3


[1] 249

## Selección de instancias y columnas

Para la selección de columnas y filas, usamos ``select`` y ``filter``. 

Todas las operaciones se pueden combinar para producir un nuevo dataset o ``SparkDataFrame``. **Son equivalentes a usar SPARKSQL **.

Estas operaciones son esenciales si queremos transformar el dataset en otra versión preprocesada del mismo.

In [5]:
# Seleccionamos sólo la columna longitud, por el id de la columna
# Por ID de columna 
head(select(df_nyctrips,df_nyctrips$pickup_longitude))

pickup_longitude
-73.94725
-73.98302
-73.99234
-73.98145
-74.00060
-74.00510


In [6]:
# Seleccionamos sólo la columna longitud, por el nombre de la columna.
# Por nombre de columna del dataset
head(select(df_nyctrips,"pickup_longitude"))

pickup_longitude
-73.94725
-73.98302
-73.99234
-73.98145
-74.00060
-74.00510


Para aplicar filtros de para las filas usamos ``filter`` que admite expresiones con operadores condicionales: 

```
    < = > ! & | ...
```

In [ ]:
# Aplicamos un filtro para ver los viajes aquellos viajes de taxi de más de 10 millas.
head(filter(df_nyctrips, df_nyctrips$trip_distance > 10 & df_nyctrips$total_amount> 20 ))

In [ ]:
# Aplicamos un filtro para ver los viajes aquellos viajes de taxi de más de 10 millas y el importe mayor de $ 20
head(filter(df_nyctrips, df_nyctrips$trip_distance > 10 & df_nyctrips$total_amount> 20 ))

In [22]:
# Aplicamos un filtro para ver el viaje más caro en Taxi que se ha hecho:
head( agg(df_nyctrips ,max = max(df_nyctrips$total_amount)))

max
70.01


In [17]:
# Aplicamos un filtro para ver el viaje menos caro en Taxi que se ha hecho:
head(agg(df_nyctrips, min = min(df_nyctrips$total_amount)))

min
3.8


## Uso de Agrupamiento y Agregación

Los SparkDataFrames soportan funciones de agregado despues de agrupar. 

Por ejemplo podemos:

In [38]:
# Agrupamos por Vendedor y mostramos el número de viajes.
head(summarize(groupBy(df_nyctrips, df_nyctrips$VendorID), count = n(df_nyctrips$VendorID)))

VendorID,count
1,100
2,149


In [52]:
# Agrupamos por Vendedor y mostramos el número de viajes.
head(summarize(groupBy(df_nyctrips, df_nyctrips$VendorID), max = max(df_nyctrips$total_amount)))

VendorID,max
1,63.50
2,70.01


In [46]:
# Agrupamos por numero de pasajeros y mostramos el numero de viajes
trips_passenger <- summarize(groupBy(df_nyctrips, df_nyctrips$passenger_count), count = n(df_nyctrips$passenger_count))

In [47]:
# Cuidado con el COLLECT !
trips_df <- head(collect(trips_passenger))

In [48]:
head(trips_df)

passenger_count,count
1,161
6,15
3,9
5,16
4,6
2,42


## Operaciones con columnas

Otras operaciones muy familiares en R, corresponden con la manipulación o transformación de valores en los registros de un dataset. En este caso la manipulación es muy sencilla:

In [53]:
# Convertimos la columna de millas a kilómetros, igual que en R.
df_nyctrips$trip_distance <- df_nyctrips$trip_distance*1.6

In [54]:
head(df_nyctrips)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
2,2016-02-25 17:24:20,2016-02-25 17:27:20,2,1.120,-73.94725,40.76377,1,N,-73.99201,40.73539,2,5.0,0.0,0.5,0.0,0,0.3,5.8
2,2016-02-25 23:10:50,2016-02-25 23:31:50,2,8.832,-73.98302,40.75099,1,N,-73.98859,40.75884,2,20.0,0.5,0.5,0.0,0,0.3,21.3
2,2016-02-01 00:00:01,2016-02-01 00:10:52,6,3.184,-73.99234,40.75820,1,N,-73.96436,40.75798,1,9.5,0.5,0.5,0.7,0,0.3,11.5
1,2016-02-01 00:00:04,2016-02-01 00:05:16,1,2.400,-73.98145,40.74972,1,N,-73.98232,40.76398,2,6.5,0.5,0.5,0.0,0,0.3,7.8
2,2016-02-01 00:00:05,2016-02-01 00:20:59,1,8.960,-74.00060,40.72976,1,N,-73.95132,40.66983,1,20.0,0.5,0.5,4.0,0,0.3,25.3
2,2016-02-01 00:00:06,2016-02-01 00:15:01,1,7.504,-74.00510,40.71901,1,N,-73.94709,40.68888,2,16.0,0.5,0.5,0.0,0,0.3,17.3


## Añadir columnas

In [57]:
# Usamos mutate para añadir columnas que operan con elementos de las demás columnas.

# mutate(sql_nyc,  uniform = rand(10),  normal  = randn(27))

head(mutate(df_nyctrips,  uniform = rand(10),  normal  = randn(27)))
head(mutate(df_nyctrips,  uniform =df_nyctrips$total_amount*1.1355,  normal  = randn(27)))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,uniform,normal
2,2016-02-25 17:24:20,2016-02-25 17:27:20,2,1.120,-73.94725,40.76377,1,N,-73.99201,...,2,5.0,0.0,0.5,0.0,0,0.3,5.8,0.41371265,0.5888539
2,2016-02-25 23:10:50,2016-02-25 23:31:50,2,8.832,-73.98302,40.75099,1,N,-73.98859,...,2,20.0,0.5,0.5,0.0,0,0.3,21.3,0.73117193,0.8645537
2,2016-02-01 00:00:01,2016-02-01 00:10:52,6,3.184,-73.99234,40.75820,1,N,-73.96436,...,1,9.5,0.5,0.5,0.7,0,0.3,11.5,0.90317012,1.2524570
1,2016-02-01 00:00:04,2016-02-01 00:05:16,1,2.400,-73.98145,40.74972,1,N,-73.98232,...,2,6.5,0.5,0.5,0.0,0,0.3,7.8,0.09430205,-2.5736369
2,2016-02-01 00:00:05,2016-02-01 00:20:59,1,8.960,-74.00060,40.72976,1,N,-73.95132,...,1,20.0,0.5,0.5,4.0,0,0.3,25.3,0.38340505,0.5469737
2,2016-02-01 00:00:06,2016-02-01 00:15:01,1,7.504,-74.00510,40.71901,1,N,-73.94709,...,2,16.0,0.5,0.5,0.0,0,0.3,17.3,0.55692461,0.1743128


VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,uniform,normal
2,2016-02-25 17:24:20,2016-02-25 17:27:20,2,1.120,-73.94725,40.76377,1,N,-73.99201,...,2,5.0,0.0,0.5,0.0,0,0.3,5.8,6.58590,0.5888539
2,2016-02-25 23:10:50,2016-02-25 23:31:50,2,8.832,-73.98302,40.75099,1,N,-73.98859,...,2,20.0,0.5,0.5,0.0,0,0.3,21.3,24.18615,0.8645537
2,2016-02-01 00:00:01,2016-02-01 00:10:52,6,3.184,-73.99234,40.75820,1,N,-73.96436,...,1,9.5,0.5,0.5,0.7,0,0.3,11.5,13.05825,1.2524570
1,2016-02-01 00:00:04,2016-02-01 00:05:16,1,2.400,-73.98145,40.74972,1,N,-73.98232,...,2,6.5,0.5,0.5,0.0,0,0.3,7.8,8.85690,-2.5736369
2,2016-02-01 00:00:05,2016-02-01 00:20:59,1,8.960,-74.00060,40.72976,1,N,-73.95132,...,1,20.0,0.5,0.5,4.0,0,0.3,25.3,28.72815,0.5469737
2,2016-02-01 00:00:06,2016-02-01 00:15:01,1,7.504,-74.00510,40.71901,1,N,-73.94709,...,2,16.0,0.5,0.5,0.0,0,0.3,17.3,19.64415,0.1743128


In [59]:
# Otro modo de hacerlo es:

head(withColumn(df_nyctrips,"uniform",rand(20)))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,uniform
2,2016-02-25 17:24:20,2016-02-25 17:27:20,2,1.120,-73.94725,40.76377,1,N,-73.99201,40.73539,2,5.0,0.0,0.5,0.0,0,0.3,5.8,0.6155304
2,2016-02-25 23:10:50,2016-02-25 23:31:50,2,8.832,-73.98302,40.75099,1,N,-73.98859,40.75884,2,20.0,0.5,0.5,0.0,0,0.3,21.3,0.4156276
2,2016-02-01 00:00:01,2016-02-01 00:10:52,6,3.184,-73.99234,40.75820,1,N,-73.96436,40.75798,1,9.5,0.5,0.5,0.7,0,0.3,11.5,0.7346618
1,2016-02-01 00:00:04,2016-02-01 00:05:16,1,2.400,-73.98145,40.74972,1,N,-73.98232,40.76398,2,6.5,0.5,0.5,0.0,0,0.3,7.8,0.9219126
2,2016-02-01 00:00:05,2016-02-01 00:20:59,1,8.960,-74.00060,40.72976,1,N,-73.95132,40.66983,1,20.0,0.5,0.5,4.0,0,0.3,25.3,0.6942660
2,2016-02-01 00:00:06,2016-02-01 00:15:01,1,7.504,-74.00510,40.71901,1,N,-73.94709,40.68888,2,16.0,0.5,0.5,0.0,0,0.3,17.3,0.2761286


### dapply -- dapplayCollect

Aplicar una función a un conjunto datos masivo con ``dapply`` y ``dapplyCollect`` 

**dapply**

Aplica una función a cada partición de un ``SparkDataFrame``. La función que será aplicada para cada partición y debería tener sólo un parámetro. La salida de la función deberá ser igualmente un data.frame. Además hay que especificar el ``schema`` del formato de los datos del ``SparkDataFrame`` resultante y deberá corresponder con tipo de datos del valor devuelto.



In [65]:
# Hacemos una copia del SparkDataFrame para usarla en una vista temporal en SQL
createOrReplaceTempView(df_nyctrips,"slqdf_filtered_nyc")

# Hacemos una selección de los registros, donde calculamos el tiempo del viaje de cada viaje
sql_nyc <- sql("select VendorID,INT(unix_timestamp(tpep_dropoff_datetime)- unix_timestamp(tpep_pickup_datetime)) AS trip_time,passenger_count,trip_distance,total_amount from slqdf_filtered_nyc")

# Mostramos un trozo de SparkDataFrame
head(sql_nyc)

schema(sql_nyc)

# Indicamos el Schema, que debe coincidir con lo que queremos
schema <- structType(
    structField("VendorID", "integer"),
    structField("trip_time", "integer"), 
    structField("passenger_count", "integer"),
    structField("trip_distance", "double"),
    structField("total_amount", "double"),
    structField("total_amount_euro", "double")
)

# Creamos la función que hará los cambios.
new_sql_nyc <- dapply(
    sql_nyc, 
    function(x) { 
        x <- cbind(x, x$total_amount*1.1355) 
    }, 
    schema)

# Vemos el cambio
head(new_sql_nyc)


VendorID,trip_time,passenger_count,trip_distance,total_amount
2,180,2,1.120,5.8
2,1260,2,8.832,21.3
2,651,6,3.184,11.5
1,312,1,2.400,7.8
2,1254,1,8.960,25.3
2,895,1,7.504,17.3


StructType
|-name = "VendorID", type = "IntegerType", nullable = TRUE
|-name = "trip_time", type = "IntegerType", nullable = TRUE
|-name = "passenger_count", type = "IntegerType", nullable = TRUE
|-name = "trip_distance", type = "DoubleType", nullable = TRUE
|-name = "total_amount", type = "DoubleType", nullable = TRUE

VendorID,trip_time,passenger_count,trip_distance,total_amount,total_amount_euro
2,180,2,1.120,5.8,6.58590
2,1260,2,8.832,21.3,24.18615
2,651,6,3.184,11.5,13.05825
1,312,1,2.400,7.8,8.85690
2,1254,1,8.960,25.3,28.72815
2,895,1,7.504,17.3,19.64415


### gapply -- gapplyCollect


Aplica una función a cada uno de los grupos de un ``SparkDataFrame``. La función será aplicada a cada grupo del ``SparkDataFrame`` y debería tener sólo dos parámetros: agrupamiento por llave y data.frame al que corresponde esa llave. La salida de la función debería ser un data.frame. 

In [ ]:
# Esquema del SparkDataFrame
schema <- structType(
    structField("VendorID", "integer"),
    structField("trip_time", "integer"), 
    structField("passenger_count", "integer"),
    structField("trip_distance", "double"),
    structField("total_amount", "double"),
    structField("max_amount", "double")
)

# Aplicamos la función gapply. Calculamos el máximo de cada Vendedor.
result <- gapply(
    sql_nyc,
    "VendorID",
    function(key, x) {
        y <- data.frame(key, max(x$total_amount))
    },
    schema)

# Mostramos el resultado.
head(result[order(result$trip_distance, decreasing = TRUE), ])



In [ ]:
head(sql_nyc)

# Ahora probamos el gapplycollect: 
# Como el gapply, aplica una funcion a cada partición y luego hace un collect del resultado en un data.frame en R.
result <- gapplyCollect(
            
    sql_nyc,
    "VendorID",
    function(key, x) {
        y <- data.frame(key, max(x$trip_distance))
        colnames(y) <- c("VendorID", "max_trip_distance")
        y
    })

# Vemos el resultado.
head(result[order(result$trip_distance, decreasing = TRUE), ])

## Operando con SparkSQL sobre cojuntos masivos de datos.

Todas las funciones de manejo de datos que se han usado con SparkR, pueden hacerse de una forma sencilla e intuitiva  con SparkSQL

In [66]:
# sql_nyc es nuestro DataFrameSpark de SQL
createOrReplaceTempView(sql_nyc,"slqdf_filtered_nyc")

# Hacemos una consulta para extraer el viaje de mayor distancia de cada venderor.
results <- sql("select VendorID, MAX(trip_distance) from slqdf_filtered_nyc GROUP BY VendorID ")

In [67]:
# Vemos el resultado.
head(results)

VendorID,max(trip_distance)
1,30.400
2,30.176


Buscamos el total de kilómetros recorridos por cada vendedor:

In [68]:
results <- sql("select VendorID, SUM(trip_distance) from slqdf_filtered_nyc GROUP BY VendorID ")

# Vemos el resultado
head(results)

VendorID,sum(trip_distance)
1,605.120
2,1054.416


Calculamos el tiempo en segundos

In [71]:
results <- sql("select VendorID, SUM(trip_time) from slqdf_filtered_nyc GROUP BY VendorID ")

# Vemos los resultados
head(results)

VendorID,sum(trip_time)
1,75020
2,187095


Calculamos el tiempo en minutos

In [73]:
results <- sql("select VendorID, SUM(trip_time)/60.0 as min_trip from slqdf_filtered_nyc GROUP BY VendorID ")

# Vemos los resultados
head(results)

VendorID,min_trip
1,1250.333
2,3118.250


Buscamos la ganacia total cada vendedor:

In [74]:
results <- sql("select VendorID, SUM(total_amount)*1.10373 as Total_Amount_Euro from slqdf_filtered_nyc GROUP BY VendorID ")

# Vemos el resultado
head(results)

VendorID,Total_Amount_Euro
1,1939.596
2,3243.752


Calculamos la media y la desviación típica del tiempo de recorrido y ganancia por numero de personas: 

In [75]:
results <- sql("select passenger_count, AVG(trip_time), AVG(total_amount) ,AVG(trip_distance)   
                from slqdf_filtered_nyc 
                GROUP BY passenger_count 
                order by passenger_count ASC ")
head(results)

passenger_count,avg(trip_time),avg(total_amount),avg(trip_distance)
1,1147.2360,17.33975,5.866236
2,877.6905,20.76762,7.781714
3,763.6667,20.23222,7.607111
4,1579.1667,39.96667,15.408000
5,722.7500,20.97500,8.040000
6,842.3333,18.31867,6.579200


Coeficiente de correlación

In [76]:
results <- sql("select corr(total_amount,trip_distance) as correlation_coef
                from 
                slqdf_filtered_nyc")
# Ver resultados
head(results)

correlation_coef
0.9101879


In [78]:
results <- sql("select corr(total_amount,trip_time) as correlation_coef
                from 
                slqdf_filtered_nyc")
head(results)

correlation_coef
0.05847827


In [79]:
results <- sql("select corr(trip_time,trip_distance) as correlation_coef
                from 
                slqdf_filtered_nyc")
head(results)

correlation_coef
0.07503141


**¿ Qué deducimos de estos coeficiente de corelación ?**

## Uso de magittr para el trabajo con los datos

El paquete magrittr permite: 

* mejorar el tiempo de desarrollo y 
* mejorar enormemente la legibilidad y mantenibilidad del código. 

Para usarlo hay que importar la biblioteca magrittr dentro del proyecto y apartir de ese momentos podemos utilizar el operador 

```
%>%
``` 

para concaternar operaciones y poder trabajar con flujos de datos y pipelines.

Provee de un operador que sirve para hacer `pipes` con el cual se puede `encauzar` un valor hacia adelante dentro de una expresión o llamada a función.

Veamos todas las operaciones que hemos realizado sobre los datos y su equivalente con `pipes`.

In [80]:
# Hacemos una copia del SparkDataFrame para usarla en una vista temporal en SQL
createOrReplaceTempView(df_nyctrips,"slqdf_filtered_nyc")

# Hacemos una selección de los registros, donde calculamos el tiempo del viaje de cada viaje
sql_nyc <- sql("select VendorID,INT(unix_timestamp(tpep_dropoff_datetime)- unix_timestamp(tpep_pickup_datetime)) AS trip_time,passenger_count,trip_distance,total_amount from slqdf_filtered_nyc")

head(sql_nyc)


VendorID,trip_time,passenger_count,trip_distance,total_amount
2,180,2,1.120,5.8
2,1260,2,8.832,21.3
2,651,6,3.184,11.5
1,312,1,2.400,7.8
2,1254,1,8.960,25.3
2,895,1,7.504,17.3


In [120]:
# Usamos magrittr
library(magrittr)

# results <- sql("select VendorID, MAX(trip_distance) from slqdf_filtered_nyc GROUP BY VendorID ")
#summarize(groupBy(df_nyctrips, df_nyctrips$passenger_count), count = n(df_nyctrips$passenger_count))

df_nyctrips %>% 
        groupBy( df_nyctrips$passenger_count) %>%
        summarize(count = n(df_nyctrips$passenger_count)) %>%
        head()


df_nyctrips %>% 
        groupBy( df_nyctrips$passenger_count) %>%
        summarize(min = min(df_nyctrips$trip_distance),max = max(df_nyctrips$trip_distance)) %>%
        head()
        

passenger_count,count
1,161
6,15
3,9
5,16
4,6
2,42


passenger_count,max
1,0.000
6,1.792
3,0.000
5,1.136
4,5.168
2,0.464


passenger_count,min,max,se
1,0.000,30.096,161
6,1.792,19.360,15
3,0.000,29.856,9
5,1.136,29.648,16
4,5.168,30.176,6
2,0.464,30.400,42


In [124]:
count(sql_nyc)
num_regs <- as.integer(count(sql_nyc))

# Mostramos el número de registros
print(num_regs)

[1] 249

[1] 249


** ¿Qué es lo mejor: `pipes`, SPARKSQL o funciones? **

<HR>
# Zona de pruebas del NOTEBOOK en SparkR
![FooterSparkR](https://sites.google.com/site/manuparra/home/footer_SparkR_v2.png)


Escribe todas las pruebas en R que necesites a partir de aquí

<HR>